Step 1: Extract and Preprocess Text from PDF

In [1]:
import fitz  # PyMuPDF
import json
from sentence_transformers import SentenceTransformer

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load and preprocess the corpus
pdf_path = r'Corpus.pdf'
corpus_text = extract_text_from_pdf(pdf_path)

# Split corpus into meaningful chunks
corpus_chunks = corpus_text.split('\n\n')  # Split based on paragraphs or other delimiters

# Load SentenceTransformer model for embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = model.encode(corpus_chunks, convert_to_tensor=True)


e:\Company\ComputerHub\wbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Step 2: Load the Q&A Data from saq.json

In [2]:
# Load Q&A data from JSON file
with open(r'SAQ.json', 'r') as f:
    saq_data = json.load(f)


Step 3: Implement the Chatbot with Streamlit

In [7]:
import streamlit as st
import openai
import torch
from sentence_transformers import SentenceTransformer, util
import json
import fitz  # PyMuPDF

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load and preprocess the corpus
pdf_path = r'Corpus.pdf'
corpus_text = extract_text_from_pdf(pdf_path)
corpus_chunks = corpus_text.split('\n\n')  # Split based on paragraphs or other delimiters

# Load SentenceTransformer model and precompute embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = model.encode(corpus_chunks, convert_to_tensor=True)

# Load Q&A data from JSON file
with open(r'SAQ.json', 'r') as f:
    saq_data = json.load(f)

# Set OpenAI API key
openai.api_key = 'your-api-key'

# Initialize session state and cache
if 'history' not in st.session_state:
    st.session_state['history'] = []
if 'cache' not in st.session_state:
    st.session_state['cache'] = {}

# Function to retrieve the most relevant answer from the corpus
def retrieve_answer(question):
    question_embedding = model.encode(question, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(question_embedding, corpus_embeddings)[0]
    top_result = torch.topk(cos_scores, k=1)
    return corpus_chunks[top_result[1].item()]

# Function to generate response
def generate_response(question):
    # Check cache first
    if question in st.session_state['cache']:
        return st.session_state['cache'][question]
    
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=question,
        max_tokens=150
    )
    answer = response.choices[0].text.strip()
    
    # Cache the response
    st.session_state['cache'][question] = answer
    return answer

# Streamlit UI
st.title("Wine Business Chatbot")
st.write("Ask me anything about our wines!")

user_input = st.text_input("You: ")

if user_input:
    st.session_state['history'].append({"role": "user", "content": user_input})

    # Check if the question matches any in the sample Q&A
    saq_answer = next((item['answer'] for item in saq_data if item['question'].lower() in user_input.lower()), None)
    
    if saq_answer:
        answer = saq_answer
    else:
        retrieved_answer = retrieve_answer(user_input)
        if retrieved_answer:
            answer = generate_response(retrieved_answer)
        else:
            answer = "Please contact the business directly for this information."
    
    st.session_state['history'].append({"role": "bot", "content": answer})
    for msg in st.session_state['history']:
        st.write(f"{msg['role']}: {msg['content']}")



In [8]:
"answer": "Jessup Cellars has a casual and inviting atmosphere and was the first tasting room opened in Yountville in 2003. 
You have the option of sitting inside our stunning art gallery or you may choose to enjoy the patio with giant umbrellas.
We also have space available for private groups and special accomodations and snacks for your children. 
Our fine art is meticulously curated by our lead artist Jermaine Dante who exhibits his colorful creations in large formats in our spacious gallery where you can take in,
or take home the inspiring art while imbibing your favorite Jessup wines."
    },

SyntaxError: unterminated string literal (detected at line 1) (1237628973.py, line 1)